In [1]:

import numpy as np

G=2
S=1
T=3
C=12




def succALAP(x,n):
    tasktype=x[0]
    i=x[1]
    j=x[2]
    k=x[3]
    listsucc=[]
    if tasktype==0:
        if i>1:
            y=(1,i,i-1,0)
            listsucc.append(y)
        else:
            y=(4,0,0,0)
            listsucc.append(y)            
    if tasktype==1:
        if j >=2:
            y=(1,i,j-1,0)
            listsucc.append(y)
        y=(2,j,i,0)
        listsucc.append(y)       
    if tasktype==2:
        y=(0,i,0,0)
        listsucc.append(y)
        if i>1:
            y=(3,i,j,i-1)
            listsucc.append(y)
    if tasktype==3:
        if k>1:
            y=(3,i,j,k-1)
            listsucc.append(y)
        y=(2,k,j,0) 
        listsucc.append(y)
        y=(2,k,i,0) 
        listsucc.append(y)

    return listsucc



# task type 0: POTRF 1:SYRK 2:TRSM 3: GEMMS
# all have 4 values, but the last one is used for GEMMS only (-;
# a task is a 4-uple and everything will be stored in dictionnaries later
# succalap is the set of successors in the alap schedule, ie the predecessors in the original graph



def succASAP(x,n):
    tasktype=x[0]
    i=x[1]
    j=x[2]
    k=x[3]
    listsucc=[]
    if tasktype==0:
        if i<n:
            for j in range(i+1,n+1,1):
                y=(2,i,j,0)
                listsucc.append(y)
        else:
            y=(4,0,0,0)
            listsucc.append(y)  
    
    
    if tasktype==1:
        if j<i-1:
            y=(1,i,j+1,0)
            listsucc.append(y)
        else:
            y=(0,i,0,0)
            listsucc.append(y)

           
            
    if tasktype==2:
        if i<=n-1:
            for k in range(i+1,j):
                y=(3,k,j,i)
                listsucc.append(y)
            for k in range(j+1,n+1):
                y=(3,j,k,i)
                listsucc.append(y)
            y=(1,j,i,0)
            listsucc.append(y)


    if tasktype==3:
        if k<i-1:
            y=(3,i,j,k+1)
            listsucc.append(y)
        else:
            y=(2,i,j,0)
            listsucc.append(y)


    return listsucc


# duration is just the duration of a task

def duration(x):
    tasktype=x[0]
    if tasktype==0:
        duration=C
    if tasktype==1:
        duration=S        
    if tasktype==2:
        duration=T
    if tasktype==3:
        duration=G
    return duration

durations = [C, S, T, G]





In [2]:
def LengthASAP(n,P):

    maxi=(G+T+S+C)*n*n*n
    mini=-1


    #ReadyTasks is the set of tasks that are ready to be processed (all their predecessors have been processed)
    # due to limited resources, they might be waiting for resources
    ReadyTasks=[]
    ReadyTasks.append((0,1,0,0))


    #Ltasks is the list of tasks that have not been processed and that are not yet ready
    LTasks=[]

    #disctionnary Processed provides, for each task, where and when it has been processed
    Processed={}
    #dictionnary Released provides, for each task, the minimum time when it can be processed (-1 at the beginning), 
    # given its already scheduled predecessors. The number of remaining predecessors to be 
    # scheduled is stored in NbSuccASAP={}
    Released={}
    NbPredASAP={}
    # Prio is the priority of the task for the alap.
    # at the moment, Prio is just the length of the critical path in ASAP (because I had the values at hand !)
    Prio={}

    CP=C+(n-1)*(T+S+C)

    Released[(0,1,0,0)]=0
    NbPredASAP[(0,1,0,0)]=0
    Prio[(0,1,0,0)]=CP

    # initialization of Prio (critical path in ASAP), Released (-1 at the beginning), 
    # NbSuccASAP (predecessors in ALAP) 

    for i in range(1,n+1):
        Released[(0,i,0,0)]=mini
        Prio[(0,i,0,0)]=C+(n-i)*(T+S+C)
    #    Prio[(0,i,0,0)]=n-i
    #    Prio[(0,i,0,0)]=i

        NbPredASAP[(0,i,0,0)]=1
        LTasks.append((0,i,0,0))


    for j in range(1,n):
        for i in range(j+1,n+1):
            Released[(1,i,j,0)]=mini
            LTasks.append((1,i,j,0))
            if i < n:
                Prio[(1,i,j,0)]=(i-j)*S+C+(n-i)*(T+S+C)
    #            Prio[(1,i,j,0)]=n-j
    #            Prio[(1,i,j,0)]=j            
                if j>1:
                    NbPredASAP[(1,i,j,0)]=2
                else:
                    NbPredASAP[(1,i,j,0)]=1

            else:
                Prio[(1,i,j,0)]=(i-j)*S+C+(n-i)*(T+S+C)
    #            Prio[(1,i,j,0)]=n-j 
    #            Prio[(1,i,j,0)]=j 
                if j>1:
                    NbPredASAP[(1,i,j,0)]=2
                else:
                    NbPredASAP[(1,i,j,0)]=1

                    
    for i in range(1,n):
        for j in range(i+1,n+1):
            LTasks.append((2,i,j,0))
            Released[(2,i,j,0)]=mini
            Prio[(2,i,j,0)]=(j-i-1)*(T+G)+(n-j+1)*(T+S+C)
    #        Prio[(2,i,j,0)]=n-i
    #        Prio[(2,i,j,0)]=i
            if i>1:
                NbPredASAP[(2,i,j,0)]=2
            else:
                NbPredASAP[(2,i,j,0)]=1

                



    for k in range(1,n-1):
        for i in range(k+1,n):
            for j in range(i+1,n+1):
                LTasks.append((3,i,j,k))
                Released[(3,i,j,k)]=mini
                Prio[(3,i,j,k)]=(i-k)*G+(j-i-1)*(T + G)+(n-j +1)*(T +S +C)
     #           Prio[(3,i,j,k)]=n-k
    #            Prio[(3,i,j,k)]=k
                if k>1:
                    NbPredASAP[(3,i,j,k)]=3
                else:
                    NbPredASAP[(3,i,j,k)]=2

    Released[(0,1,0,0)]=0
    NbPredASAP[(0,1,0,0)]=0
    Prio[(0,1,0,0)]=CP

#    print("Prio",Prio)



                
    #ProcReady[i] is the time when processor P_i is available.

    ProcReady=[0]*P



    # instant represents the current time
    instant=0

    while len(ReadyTasks)>0:

        for i in range(P):
            ProcReady[i]=max(instant,ProcReady[i])
#    print('debut',"\n",instant)
#    print('ReadyTasks',ReadyTasks)
#    print("Processed",Processed)
#    print('ProcReady',ProcReady,"\n\n\n")

#indices is the list of the processors that are ready the soonest
        indices = [i for i, x in enumerate(ProcReady) if x == min(ProcReady)]
        
#nextinstant1 is the next instant when something can happen         

        for i in indices:
        # first find the highest priority task among the ready ones
            if (len(ReadyTasks)>0):
                HighestPrio=mini
                for task in ReadyTasks:
                    if Prio[task] >= HighestPrio:
                        if Released[task]<=instant:
                            if min(ProcReady)<=instant:
                                HighestPrio = Prio[task]
                                HPTask=task
#                print("instant",instant,"HPTASK",HPTask,"ReadyTasks",ReadyTasks)
            #mark all its successors with one depedendency less        
                for task in succASAP(HPTask,n):
                    Released[task]=max(Released[task],ProcReady[i]+duration(HPTask))
                    NbPredASAP[task]=NbPredASAP[task]-1
#                print("released",Released[(0,2,0,0)],NbPredASAP[(0,2,0,0)])
#                print(instant)
#                print("ReadyTasks",ReadyTasks)
        #mark the task processed, update the availability of the associated proc and remove it from some lists    
                ProcReady[i]=max(ProcReady[i],instant)+duration(HPTask)
                Processed[HPTask]=[i,ProcReady[i]]
                ReadyTasks.remove(HPTask)
                del Released[HPTask]
        #mininstant1 is the next instant when a busy processor will be released

        nextinstant=maxi

    # if there are no more ready tasks, we should determine the time when something will happen
    # ie a task that is not ready yet but will become soon given what has been scheduled (ie no dependencies
    # but still running predecessors)
    # in this first part, the goal is to find the first instant when a task will become ready
#        if len(ReadyTasks)==0:
#            print("zaza")
        for task in ReadyTasks:
            nextinstant=min(nextinstant,Released[task])
         
        nextinstant=max(min(ProcReady),nextinstant)   
        
        for task in LTasks:
            if task not in ReadyTasks:
                if NbPredASAP[task] ==0:
                    if task in Released:
                        if Released[task] <= nextinstant:
                            nextinstant=min(nextinstant,Released[task])
#        nextinstant=min(ProcReady,nextinstant)
#            print("zouzou")

   # once we now this instant (nextinstant) just put the corresponding task(s) into the ready list                       
        for task in LTasks:
            if task not in ReadyTasks:
                if NbPredASAP[task] ==0:
                    if task in Released:
                        if Released[task] <= nextinstant:                       
                            ReadyTasks.append(task)


   
    #and advance the time until the new instant when something will happen (and in particlar there
    #should be a ready task)


        instant=max(min(ProcReady),nextinstant)
#        print("instant a la mise a jour",instant)


        
        if (0,n,0,0) in ReadyTasks:
            #we reached the first POTRF
            break


            
    Processed[(0,n,0,0)]=[0,max(ProcReady)+C]  

#    print(Processed)
#    print("CP=",CP,"endtime=",max(ProcReady)+C)
                
    for key in Processed:
        Processed[key][1]=max(ProcReady)+C-Processed[key][1]+duration(key)
        
#    print(Processed)
    return(Processed,max(ProcReady)+C)
    


In [3]:
def LengthALAP(n,P):

    maxi=(G+T+S+C)*n*n*n
    mini=-1



    #ReadyTasks is the set of tasks that are ready to be processed (all their predecessors have been processed)
    # due to limited resources, they might be waiting for resources
    ReadyTasks=[]
    ReadyTasks.append((0,n,0,0))


    #Ltasks is the list of tasks that have not been processed and that are not yet ready
    LTasks=[]

    #disctionnary Processed provides, for each task, where and when it has been processed
    Processed={}
    #dictionnary Released provides, for each task, the minimum time when it can be processed (-1 at the beginning), 
    # given its already scheduled predecessors. The number of remaining predecessors to be 
    # scheduled is stored in NbSuccASAP={}
    Released={}
    NbSuccASAP={}
    # Prio is the priority of the task for the alap.
    # at the moment, Prio is just the length of the critical path in ASAP (because I had the values at hand !)
    Prio={}

    CP=C+(n-1)*(T+S+C)

    Released[(0,n,0,0)]=mini
    NbSuccASAP[(0,n,0,0)]=0
    Prio[(0,n,0,0)]=0

# initialization of Prio (critical path in ASAP), Released (-1 at the beginning), 
# NbSuccASAP (predecessors in ALAP) 

    for i in range(1,n):
        Released[(0,i,0,0)]=mini
        Prio[(0,i,0,0)]=CP-(i-1)*(C+S+T)
#    Prio[(0,i,0,0)]=n-i
#    Prio[(0,i,0,0)]=i

        NbSuccASAP[(0,i,0,0)]=n-i
        LTasks.append((0,i,0,0))



    for j in range(1,n):
        for i in range(j+1,n+1):
            Released[(1,i,j,0)]=mini
            NbSuccASAP[(1,i,j,0)]=1
            LTasks.append((1,i,j,0))
            if i < n:
                Prio[(1,i,j,0)]=CP-(j-1)*(C+S+T)-C-T
 #           Prio[(1,i,j,0)]=n-j
#            Prio[(1,i,j,0)]=j



            else:
                Prio[(1,i,j,0)]=CP-(j-1)*(C+S+T)-C-T
#            Prio[(1,i,j,0)]=n-j 
#            Prio[(1,i,j,0)]=j 


    for i in range(1,n):
        for j in range(i+1,n+1):
            LTasks.append((2,i,j,0))
            Released[(2,i,j,0)]=mini
            Prio[(2,i,j,0)]=CP-(i-1)*(C+S+T)-C
#        Prio[(2,i,j,0)]=n-i
#        Prio[(2,i,j,0)]=i


            NbSuccASAP[(2,i,j,0)]=n-i




    for k in range(1,n-1):
        for i in range(k+1,n):
            for j in range(i+1,n+1):
                LTasks.append((3,i,j,k))
                Released[(3,i,j,k)]=mini
                Prio[(3,i,j,k)]=CP-(k-1)*(C+S+T)-C-T
 #           Prio[(3,i,j,k)]=n-k
#            Prio[(3,i,j,k)]=k
                NbSuccASAP[(3,i,j,k)]=1




            
#ProcReady[i] is the time when processor P_i is available.

    ProcReady=[0]*P



# instant represents the current time



    # instant represents the current time
    instant=0

    while len(ReadyTasks)>0:

        for i in range(P):
            ProcReady[i]=max(instant,ProcReady[i])
#    print('debut',"\n",instant)
#    print('ReadyTasks',ReadyTasks)
#    print("Processed",Processed)
#    print('ProcReady',ProcReady,"\n\n\n")

#indices is the list of the processors that are ready the soonest
        indices = [i for i, x in enumerate(ProcReady) if x == min(ProcReady)]
        
#nextinstant1 is the next instant when something can happen         

        for i in indices:
        # first find the highest priority task among the ready ones
            if len(ReadyTasks)>0:
                HighestPrio=maxi
                for task in ReadyTasks:
                    if Prio[task] <= HighestPrio:
                        if Released[task]<=instant:
                            if min(ProcReady)<=instant:                    
                                HighestPrio = Prio[task]
                                HPTask=task
            #mark all its successors with one depedendency less        
                for task in succALAP(HPTask,n):
                    Released[task]=max(Released[task],ProcReady[i]+duration(HPTask))
                    NbSuccASAP[task]=NbSuccASAP[task]-1

        #mark the task processed, update the availability of the associated proc and remove it from some lists    
                ProcReady[i]=max(ProcReady[i],instant)+duration(HPTask)
                Processed[HPTask]=[i,ProcReady[i]]
                ReadyTasks.remove(HPTask)
                del Released[HPTask]
        #mininstant1 is the next instant when a busy processor will be released

        nextinstant=maxi

    # if there are no more ready tasks, we should determine the time when something will happen
    # ie a task that is not ready yet but will become soon given what has been scheduled (ie no dependencies
    # but still running predecessors)
    # in this first part, the goal is to find the first instant when a task will become ready
        for task in ReadyTasks:
            nextinstant=min(nextinstant,Released[task])    
        nextinstant=max(min(ProcReady),nextinstant)   

        
        for task in LTasks:
            if task not in ReadyTasks:
                if NbSuccASAP[task] ==0:
                    if task in Released:
                        if Released[task] <= nextinstant:
                            nextinstant=min(nextinstant,Released[task])


   # once we now this instant (nextinstant) just put the corresponding task(s) into the ready list                       
        for task in LTasks:
            if task not in ReadyTasks:
                if NbSuccASAP[task] ==0:
                    if task in Released:
                        if Released[task] <= nextinstant:                       
                            ReadyTasks.append(task)

    
    #and advance the time until the new instant when something will happen (and in particlar there
    #should be a ready task)


        instant=max(min(ProcReady),nextinstant)
        
        if (0,1,0,0) in ReadyTasks:
            #we reached the first POTRF
            break


            
    Processed[(0,1,0,0)]=[0,max(ProcReady)+C]  

#    print(Processed)
#    print("CP=",CP,"endtime=",max(ProcReady)+C)                

    return(Processed,max(ProcReady)+C)
    
  


In [6]:
LengthASAP(8,4)


({(0, 1, 0, 0): [0, 124],
  (2, 1, 2, 0): [0, 112],
  (2, 1, 3, 0): [1, 112],
  (2, 1, 4, 0): [2, 112],
  (2, 1, 5, 0): [3, 112],
  (1, 2, 1, 0): [0, 109],
  (3, 2, 3, 1): [1, 109],
  (1, 3, 1, 0): [2, 109],
  (3, 2, 4, 1): [3, 109],
  (0, 2, 0, 0): [0, 108],
  (3, 3, 4, 1): [2, 108],
  (1, 4, 1, 0): [1, 107],
  (3, 2, 5, 1): [3, 107],
  (3, 3, 5, 1): [1, 106],
  (3, 4, 5, 1): [2, 106],
  (2, 1, 6, 0): [3, 105],
  (1, 5, 1, 0): [1, 104],
  (2, 1, 7, 0): [2, 104],
  (2, 1, 8, 0): [1, 103],
  (3, 2, 6, 1): [3, 102],
  (3, 3, 6, 1): [2, 101],
  (3, 4, 6, 1): [1, 100],
  (3, 5, 6, 1): [3, 100],
  (3, 2, 7, 1): [2, 99],
  (3, 3, 7, 1): [1, 98],
  (1, 6, 1, 0): [3, 98],
  (3, 4, 7, 1): [2, 97],
  (3, 5, 7, 1): [3, 97],
  (2, 2, 3, 0): [0, 96],
  (2, 2, 4, 0): [1, 96],
  (2, 2, 5, 0): [2, 95],
  (2, 2, 6, 0): [3, 95],
  (1, 3, 2, 0): [0, 93],
  (3, 3, 4, 2): [1, 93],
  (0, 3, 0, 0): [0, 92],
  (1, 4, 2, 0): [2, 92],
  (3, 3, 5, 2): [3, 92],
  (3, 4, 5, 2): [1, 91],
  (1, 5, 2, 0): [2, 91],
  